In [34]:
anki_collection_path = '/mnt/c/users/ambru/AppData/Roaming/Anki2/User 1/collection.anki2'
deck_name = "my-danish-words"
model_name = "two-lang-with-audio"

In [35]:
from anki.storage import Collection

class AnkiCollectionManager:
    def __init__(self, collection_path):
        self.collection_path = collection_path
        self.collection = None

    def __enter__(self):
        # Open the collection and return it
        self.collection = Collection(self.collection_path)
        return self.collection

    def __exit__(self, exc_type, exc_val, exc_tb):
        # Ensure the collection is closed
        if self.collection:
            self.collection.close()
            self.collection = None

In [36]:
def get_deck_and_model_ids(collection_path, deck_name):
    with AnkiCollectionManager(collection_path) as col:
        # Get deck names and IDs
        deck_data = col.decks.all_names_and_ids()
        deck_id = None
        print(deck_data)
        print(type(deck_data))
        for d in deck_data:
            name = d.name
            id = d.id
            if name == deck_name:
                deck_id = id
                break
        
        if not deck_id:
            print(f"Deck '{deck_name}' not found.")
            return

        print(f"Deck ID for '{deck_name}': {deck_id}")

        # Get model names and IDs
        model_data = col.models.all_names_and_ids()
        print("Models and their IDs:")
        for m in model_data:
            model_name = m.name
            model_id = m.id
            print(f"Model Name: {model_name}, Model ID: {model_id}")

        # Return deck_id and model_ids for further use
        return deck_id, model_data

# Replace with the path to your Anki collection file
collection_path = '/mnt/c/users/ambru/AppData/Roaming/Anki2/User 1/collection.anki2'
deck_name = "my-danish-words"
deck_id, model_ids = get_deck_and_model_ids(collection_path, deck_name)


[id: 1717014483954
name: "Danish 1000 with pronunciation"
, id: 1717014467832
name: "Danish 8000 Sentences Revamped"
, id: 1717014475428
name: "Danish Core 100"
, id: 1
name: "Default"
, id: 1725308120368
name: "my-danish-words"
]
<class 'google._upb._message.RepeatedCompositeContainer'>
Deck ID for 'my-danish-words': 1725308120368
Models and their IDs:
Model Name: 0 Neri's Sentences (Read), Model ID: 1611000932419
Model Name: 0 Neri's Sentences (Speak), Model ID: 1611291012310
Model Name: Basic, Model ID: 1667341440148
Model Name: Basic (and reversed card), Model ID: 1667341440149
Model Name: Basic (optional reversed card), Model ID: 1667341440150
Model Name: Basic (type in the answer), Model ID: 1667341440151
Model Name: Basic+, Model ID: 1564286212557
Model Name: Basic-0dc55, Model ID: 1588793888206
Model Name: Basic-1df79, Model ID: 1472105227674
Model Name: Basic-2e165, Model ID: 1436153614454
Model Name: Basic-835b2, Model ID: 1426108799663
Model Name: Cloze, Model ID: 1667341440

In [55]:
def create_model_with_audio(collection_path, model_name, deck_name):
    with AnkiCollectionManager(collection_path) as col:
        # Define the model
        model = {
            'name': model_name,
            'flds': [
                {'name': 'Word'},
                {'name': 'Sentence'},
                {'name': 'Translation'},
                {'name': 'AudioWord'},
                {'name': 'AudioSentence'}
            ],
            'tmpls': [
                {
                    'name': 'Card 1',
                    'qfmt': '{{Word}}<br>{{Sentence}}<br>{{AudioWord}}',
                    'afmt': '{{FrontSide}}<hr id="answer">{{Translation}}<br>{{AudioSentence}}'
                },
                {
                    'name': 'Card 2',
                    'qfmt': '{{Translation}}<br>{{AudioSentence}}',
                    'afmt': '{{FrontSide}}<hr id="answer">{{Sentence}}<br>{{AudioWord}}'
                }
            ],
            'tags': [],
            'modelType': 1  # Basic model type; adjust as needed
        }

        # Add the new model to the collection
        model_id = col.models.add_dict(model)
        
        # Create the deck if it does not exist
        deck_id = col.decks.id(deck_name)
        if not deck_id:
            deck_id = col.decks.add(deck_name)
        
        # Add model to the deck
        col.decks.select(deck_id)
        col.models.add_to_deck(model_id, deck_id)
        
        # Save changes
        col.save()

create_model_with_audio(collection_path, model_name, deck_name)

BackendError: JsonError { info: "missing field `sticky` at line 1 column 53" }

In [ ]:
def create_model_with_audio(collection_path, model_name, deck_name):
    with AnkiCollectionManager(collection_path) as col:
        # Define new model fields
        fields = [
            {'name': 'Word'},                  # Field for the word
            {'name': 'Sentence'},              # Field for the sentence
            {'name': 'WordOtherLang'},         # Field for the word in the other language
            {'name': 'SentenceOtherLang'},     # Field for the sentence in the other language
            {'name': 'AudioFront'},            # Field for audio on the front side
            {'name': 'AudioBack'}              # Field for audio on the back side
        ]

        # Define card templates
        templates = [
            {
                'name': 'Card 1',
                'qfmt': '{{Word}}<br>{{Sentence}}<br>{{AudioFront}}',  # Question format (front of the card)
                'afmt': '{{FrontSide}}<hr id="answer">{{WordOtherLang}}<br>{{SentenceOtherLang}}<br>{{AudioBack}}',  # Answer format (back of the card)
            }
        ]

        # Define the new model
        new_model = {
            'name': model_name,
            'flds': fields,             # Fields for the model
            'tmpls': templates,         # Templates for the model
            'sticky': False,            # Add the missing 'sticky' field
            'tags': [],                 # Add an empty 'tags' field
            'did': None,                # Add a 'did' field for deck ID
            'css': '',                  # Add an empty 'css' field
            'req': []                   # Add an empty 'req' field
        }

        # Add the new model
        model_id = col.models.add(new_model)
        print(f"Created new model '{model_name}' with ID: {model_id}")

        # Create or get the deck ID
        deck_id = col.decks.id(deck_name)
        if deck_id is None:
            deck_id = col.decks.add(deck_name)
            print(f"Created new deck '{deck_name}' with ID: {deck_id}")

        # Add the new model to the deck
        col.decks.add_model(deck_id, model_id)
        print(f"Added model '{model_name}' to deck '{deck_name}'")

        return model_id, deck_id


create_model_with_audio(collection_path, model_name, deck_name)